In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 25.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.linalg import eigvals
from Bio import SeqIO
from scipy.linalg import eig




# ME: Calculates the eigenvalue-based metric from matrix W
def ME(W):
    if W.ndim == 1:
        # If W is a 1D array, reshape it to 2D
        W = W.reshape(-1, 1)

    W = W[1:, :]  # Remove the first row
    D = pdist(W)  # Pairwise distances
    E = squareform(D)

    x = W.shape[0]
    sdist = np.zeros((x, x))

    for i in range(x):
        for j in range(i, x):
            if j - i == 1:
                sdist[i, j] = E[i, j]
            elif j - i > 1:
                sdist[i, j] = sdist[i, j - 1] + E[j - 1, j]

    sd = sdist + sdist.T
    sdd = sd + np.eye(x)
    L = E / sdd

    # Get the eigenvalues and find the largest
    eigvals = eig(L, right=False)
    largest_eigval = np.max(eigvals.real)  # Use the real part of the eigenvalues

    return largest_eigval / x



# GRS: Calculates the geometric representation of the sequence in space
def GRS(seq, P, V, M):
    l_seq = len(seq)
    k = M.shape[0]

    g = []
    for j in range(k):
        c = np.zeros(3)
        d = np.zeros(3)
        y = np.zeros(20)

        for i in range(l_seq):
            x = (seq[i] == M[j, :])

            if i == 0:
                c = c + x.dot(P)
            elif all(x == 0):
                d = d * (i - 1) / i
                c = c + np.array([0, 0, 1]) + d
            elif all(y == 0):
                d = d * (i - 1) / i
                c = c + x.dot(P) + d
            else:
                d = d * (i - 1) / i + V[np.where(y == 1)[0][0], np.where(x == 1)[0][0]] / i
                c = c + x.dot(P) + d

            y = x

        g.append(c)

    return np.array(g)




# FEGS: Extract features from sequences in a FASTA file
def FEGS(sequences):
    P, V = coordinate()

    # Load sequences from the FASTA file
    # sequences = [str(record.seq) for record in SeqIO.parse(fasta_file, "fasta")]
    l = len(sequences)

    # Initialize results
    g_p = []
    EL = np.zeros((l, 158))
    FA = np.zeros((l, 20))
    FD = np.zeros((l, 400))
    char = "ARNDCQEGHILKMFPSTWYV"

    # Parallel processing setup (optional, can use joblib or multiprocessing)
    for i in range(l):
        # Calculate GRS and ME features
        g_p.append(GRS(sequences[i], P, V, np.array([list(char)] * 158)))

        for u in range(158):
            EL[i, u] = ME(g_p[i][u])

        # Calculate AAC and DPC features
        AAC, DPC = SAD(sequences[i], char)
        FA[i, :] = AAC
        FD[i, :] = DPC.flatten()

    # Combine all features
    FV = np.hstack([EL, FA, FD])

    return FV

#